In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import os
import math

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
c=0
#let's create datasets without overlapping
#a bit of dataset

def remove_duplicates(x1, x2, y1):
    counter=0
    to_delete=[-1]
    print(y1.shape)
    l1 = len(x1)
    l2 = len(x2)
    for i in range(l1):
        for j in range(l2):            
            if (np.sum(np.abs(x1[i,:]-x2[j,:]))==0):
                if (to_delete[-1]!=i):
                    to_delete = np.append(to_delete, i)
                    #print(to_delete)
                counter+=1
                
    to_delete = np.delete(to_delete, 0)        
    x1 = np.delete(x1, to_delete, 0)
    y1 = np.delete(y1, to_delete, 0)
    print(x1.shape, y1.shape)
    return (counter, x1, y1)

c, test_c, test_lc = remove_duplicates(test_dataset, train_dataset, test_labels)
print(c)


(10000, 10)


KeyboardInterrupt: 

In [39]:
pickle_file = os.path.join('.', 'notMNIST.pickle')

try:
    f = open(pickle_file, 'wb')
    save = {
        'test_dataset': test_c, #русская с, чтоб ее!
        'test_labels': test_lc,
        }
    pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', pickle_file, ':', e)
    raise

In [5]:
test_dataset = test_c
test_labels = test_lc

NameError: name 'test_c' is not defined

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.05

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf_train_labels, logits=logits)) + beta*tf.nn.l2_loss(weights)

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

In [6]:
num_steps = 801



with tf.Session(graph=graph) as session:
      # This is a one-time operation which ensures the parameters get initialized as
      # we described in the graph: random weights for the matrix, zeros for the
      # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
                    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 171.787537
Training accuracy: 12.3%
Validation accuracy: 12.7%
Loss at step 100: 1.731257
Training accuracy: 81.9%
Validation accuracy: 80.3%
Loss at step 200: 0.870116
Training accuracy: 82.7%
Validation accuracy: 81.2%
Loss at step 300: 0.864388
Training accuracy: 82.7%
Validation accuracy: 81.2%
Loss at step 400: 0.864063
Training accuracy: 82.7%
Validation accuracy: 81.2%
Loss at step 500: 0.863939
Training accuracy: 82.7%
Validation accuracy: 81.3%
Loss at step 600: 0.863883
Training accuracy: 82.7%
Validation accuracy: 81.3%
Loss at step 700: 0.863858
Training accuracy: 82.7%
Validation accuracy: 81.3%
Loss at step 800: 0.863846
Training accuracy: 82.7%
Validation accuracy: 81.3%
Test accuracy: 88.0%


In [46]:
batch_size = 128
num_nodes = 512
beta = 0.005
    

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    

    # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_nodes]))
    b1 = tf.Variable(tf.zeros([num_nodes]))

    w2 = tf.Variable(
        tf.truncated_normal([num_nodes, num_nodes]))
    b2 = tf.Variable(tf.zeros([num_nodes]))
    
    w3 = tf.Variable(
        tf.truncated_normal([num_nodes, num_labels]))
    b3 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # not very beautiful, but I still don't understand what's wrong
    #too fast in the beginning, too fast in the end
    sumx = 0
    for i in range(7):
        x = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), 0.5)
        logitsi = tf.matmul(tf.nn.relu(tf.matmul(x, w2) + b2), w3) + b3
        sumx +=logitsi
    logits = sumx/7

    #x1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    #x2 = tf.nn.relu(tf.matmul(x1, w2) + b2)
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf_train_labels, logits=logits)) + beta*(
                tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(w3))

    # Optimizer.
    
    global_step = tf.Variable(0)
    
    #reduce = 1 - 7* (global_step/30000) #at first almost 1, at the end 0.1
    #reduce = tf.cast(reduce, tf.float32)
    #k = 0.5*(1500/(loss+1000))*reduce #little steps when loss is big, little steps when we are almost there
    k = 0.5*(1500/(loss+1000))
    learning_rate = tf.train.exponential_decay(k, global_step, 100, 0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
  
    train_prediction = tf.nn.softmax(logits)
    v1 = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    v2 = tf.nn.relu(tf.matmul(v1, w2) + b2)
    valid_prediction = tf.nn.softmax(tf.matmul(v2, w3) + b3)
        
    t1 = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    t2 = tf.nn.relu(tf.matmul(t1, w2) + b2)
    test_prediction = tf.nn.softmax(tf.matmul(t2, w3) + b3)

In [47]:
num_steps = 3001
overfit = False
k = 15

drop = 3

if overfit:
    train_d = train_dataset[:(k*128), :]
    train_l = train_labels[:(k*128), :]
else:
    train_d = train_dataset[:, :]
    train_l = train_labels[:, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_l.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_d[offset:(offset + batch_size), :]
        batch_labels = train_l[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions, = session.run(
                [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4997.351074
Minibatch accuracy: 10.9%
Validation accuracy: 20.0%
Minibatch loss at step 500: 212.150467
Minibatch accuracy: 61.7%
Validation accuracy: 50.7%
Minibatch loss at step 1000: 41.512421
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 14.849508
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 8.182546
Minibatch accuracy: 90.6%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 5.909012
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 3000: 4.937388
Minibatch accuracy: 84.4%
Validation accuracy: 84.6%
Test accuracy: 91.5%


In [29]:
batch_size = 128
num_nodes = 1024
beta = 0.001
    

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    

    # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_nodes]))
    b1 = tf.Variable(tf.zeros([num_nodes]))

    
    w2 = tf.Variable(
        tf.truncated_normal([num_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # not very beautiful, but I still don't understand what's wrong
    #x1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1), 0.5)
    x1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    logits = tf.matmul(x1, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=tf_train_labels, logits=logits)) + beta*(
                tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2))

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 100, 0.9, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
  
    train_prediction = tf.nn.softmax(logits)
    v1 = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(v1, w2) + b2)
        
    t1 = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(t1, w2) + b2)

In [30]:
num_steps = 3001
overfit = False
k = 15
if overfit:
    train_d = train_dataset[:(k*128), :]
    train_l = train_labels[:(k*128), :]
else:
    train_d = train_dataset[:, :]
    train_l = train_labels[:, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_l.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_d[offset:(offset + batch_size), :]
        batch_labels = train_l[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions, = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 686.001953
Minibatch accuracy: 3.9%
Validation accuracy: 27.1%
Minibatch loss at step 500: 216.502167
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 164.270981
Minibatch accuracy: 81.2%
Validation accuracy: 82.4%
Minibatch loss at step 1500: 140.523407
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at step 2000: 128.560913
Minibatch accuracy: 90.6%
Validation accuracy: 83.9%
Minibatch loss at step 2500: 121.913162
Minibatch accuracy: 88.3%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 118.541451
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Test accuracy: 91.0%


In [48]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

NameError: name 'num_channels' is not defined